In [1]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/sentiment/news-sentiment/sentiment-data-v2.csv

In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/sentiment/positive/strong-positives.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/sentiment/negative/strong-negatives-2.json

In [3]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/sentiment/translate/multidomain-sentiment/bm-amazon.json
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/sentiment/translate/multidomain-sentiment/bm-imdb.json
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/sentiment/translate/multidomain-sentiment/bm-yelp.json

In [4]:
from malaya.text.bpe import WordPieceTokenizer
tokenizer = WordPieceTokenizer('BERT.wordpiece', do_lower_case = False)
# tokenizer.tokenize('halo nama sayacomel')

In [5]:
import random
from unidecode import unidecode
import re

normalized_chars = {}

chars = '‒–―‐—━—-▬'
for char in chars:
    normalized_chars[ord(char)] = '-'

chars = '«»“”¨"'
for char in chars:
    normalized_chars[ord(char)] = '"'

chars = "’'ʻˈ´`′‘’\x92"
for char in chars:
    normalized_chars[ord(char)] = "'"

chars = '̲_'
for char in chars:
    normalized_chars[ord(char)] = '_'

chars = '\xad\x7f'
for char in chars:
    normalized_chars[ord(char)] = ''

chars = '\n\r\t\u200b\x96'
for char in chars:
    normalized_chars[ord(char)] = ' '

    
laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'hoho',
    'hewhew',
    'uwu',
    'sksk',
    'ksks',
    'gituu',
    'gitu',
    'mmeeooww',
    'meow',
    'alhamdulillah',
    'muah',
    'mmuahh',
    'hehe',
    'salamramadhan',
    'happywomensday',
    'jahagaha',
    'ahakss',
    'ahksk'
}

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def cleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    
    chars = '.,/'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    x = []
    for word in string:
        word = word.lower()
        if any([laugh in word for laugh in laughing]):
            if random.random() >= 0.5:
                x.append(word)
        else:
            x.append(word)
    string = [w.title() if w[0].isupper() else w for w in x]
    return ' '.join(string)

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
from unidecode import unidecode

df = pd.read_csv('sentiment-data-v2.csv')
Y = LabelEncoder().fit_transform(df.label)
texts = df.iloc[:,1].tolist()
labels = Y.tolist()

assert len(labels) == len(texts)

In [7]:
import json
with open('bm-amazon.json') as fopen:
    amazon = json.load(fopen)
    
with open('bm-imdb.json') as fopen:
    imdb = json.load(fopen)
    
with open('bm-yelp.json') as fopen:
    yelp = json.load(fopen)
    
texts += amazon['negative']
labels += [0] * len(amazon['negative'])
texts += amazon['positive']
labels += [1] * len(amazon['positive'])

texts += imdb['negative']
labels += [0] * len(imdb['negative'])
texts += imdb['positive']
labels += [1] * len(imdb['positive'])

texts += yelp['negative']
labels += [0] * len(yelp['negative'])
texts += yelp['positive']
labels += [1] * len(yelp['positive'])

In [8]:
with open('strong-positives.json') as fopen:
    positives = json.load(fopen)
    positives = random.sample(positives, 500000)
    
len(positives)

500000

In [9]:
with open('strong-negatives.json') as fopen:
    negatives = json.load(fopen)
    negatives = random.sample(negatives, 500000)
    
len(negatives)

500000

In [10]:
texts += negatives
labels += [0] * len(negatives)
texts += positives
labels += [1] * len(positives)

In [11]:
from tqdm import tqdm

for i in tqdm(range(len(texts))):
    texts[i] = cleaning(texts[i])

100%|██████████| 1006131/1006131 [01:26<00:00, 11592.50it/s]


In [12]:
actual_t, actual_l = [], []

for i in tqdm(range(len(texts))):
    if len(texts[i]) > 2:
        actual_t.append(texts[i])
        actual_l.append(labels[i])

100%|██████████| 1006131/1006131 [00:00<00:00, 1297341.78it/s]


In [13]:
from tqdm import tqdm

input_ids, input_masks = [], []

for text in tqdm(actual_t):
    tokens_a = tokenizer.tokenize(text)
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    
    input_ids.append(input_id)
    input_masks.append(input_mask)

100%|██████████| 1006130/1006130 [05:21<00:00, 3125.89it/s]


In [14]:
import pickle

with open('sentiment-fastformer.pkl', 'wb') as fopen:
    pickle.dump([input_ids, actual_l], fopen)